In [76]:
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
from time import time
from collections import defaultdict

In [271]:
base_url = 'https://dermae.com'
suffix_url = '/collections/all?page='
username = 'dermae'


In [283]:
def get_image_url(soup):
    each_product = soup.find_all('div',{'class':'grid__item small--one-whole medium--one-half large--one-fifth product-grid-item'})
    return ['https://'+x.img['src'].strip('/') for x in each_product]

def get_sku(soup):
    
    
#     if x.find('select',{'name':'id'}).find('option')['data-sku'] is not None
    each_product = soup.find_all('div',{'class':'grid__item small--one-whole medium--one-half large--one-fifth product-grid-item'})
    
    #item_details = [x for x in prod.find_all('div',class_='product details product-item-details')]

    
    
    return [x['id'] for x in each_product]    
    
    #return  [x.find('select',{'name':'id'}).find('option')['data-sku'] for x in each_product]
    

def get_price(soup):
    try:
        each_product = soup.find_all('div',{'class':'grid__item small--one-whole medium--one-half large--one-fifth product-grid-item'})

        price = [float(x.find(class_ = 'product-single__variants').text.strip().strip('$')) for x in each_product]
        
    except ValueError:
        price = [None for x in each_product]
        
    return price

def get_product_url(soup):
    each_product = soup.find_all('div',{'class':'grid__item small--one-whole medium--one-half large--one-fifth product-grid-item'})
     
    return [base_url + x.find('div',{'class':'figcaption under'}).find('a')['href'] for x in each_product]   

def get_product_name(soup):
    each_product = soup.find_all('div',{'class':'grid__item small--one-whole medium--one-half large--one-fifth product-grid-item'})
    
    return [x.find('div',class_ = 'figcaption under').find('p').text.strip() for x in each_product]

In [284]:
funcs = {'name':get_product_name,
         'product_url' : get_product_url,
         'price' : get_price,
         'sku' : get_sku,
         'image_url' : get_image_url}

In [285]:
product_dict = defaultdict(list)

for counter in range(1,4):
    try:
        r = requests.get(base_url+suffix_url+str(counter))
    except ConnectionError:
        break
        
    soup = BeautifulSoup(r.text,'lxml')

    for key,func in funcs.items():
        product_dict[key].extend(func(soup))
    
    
# Creating additional columns
product_dict['username'] = [username for x in range(len(product_dict['name']))]
product_dict['main_category'] = [None for x in range(len(product_dict['name']))]
product_dict['sub_category'] = [None for x in range(len(product_dict['name']))]
product_dict['product_category'] = [None for x in range(len(product_dict['name']))]


In [286]:
df = pd.DataFrame(product_dict)
df.head()

,image_url,main_category,name,price,product_category,product_url,sku,sub_category,username
0,https://cdn.shopify.com/s/files/1/1525/1400/pr...,None,$20 Gift Card,None,None,https://dermae.com/products/gift-card,10815783368,None,dermae
1,https://cdn.shopify.com/s/files/1/1525/1400/pr...,None,$40 Gift Card,None,None,https://dermae.com/products/40-gift-card,60826943496,None,dermae
2,https://cdn.shopify.com/s/files/1/1525/1400/pr...,None,3-in-1 Hair Protection Serum,None,None,https://dermae.com/products/3-in-1-hair-protec...,10079341576,None,dermae
3,https://cdn.shopify.com/s/files/1/1525/1400/pr...,None,Age-Defying Antioxidant Day Cream,None,None,https://dermae.com/products/age-defying-antiox...,7966871368,None,dermae
4,https://cdn.shopify.com/s/files/1/1525/1400/pr...,None,Age-Defying Antioxidant Eye Cream,None,None,https://dermae.com/products/age-defying-antiox...,7967103368,None,dermae


In [289]:
# Dropping duplicates

df2 = df.drop_duplicates(subset = ['name','sku'])
df2.shape

(100, 9)

In [290]:
# Exporting to csv

df2.to_csv(username+'.csv')